In [ ]:
#As always, we import everything
import os
import pandas as pd
import json
import folium
import math
%matplotlib inline
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import base64

First, let's get the data and put it in a dataframe.

In [ ]:
eu_data = pd.read_table('tsdec450.tsv', na_values=': ')

Now, we can process it to get the id for each country.

In [ ]:
eu_data['info'] = eu_data['info'].apply(lambda x: x.split(",")[3])
eu_data

Now we can define a function that returns the color of a country based on its unemployement rate.
For this, we need to choose a palette of colors : we used colorbrewer to get one which is color-blind friendly.
To get the color for a country, we simply bin all the unemployement_rate into one of the colors.

In this function, we also create a mapping from country id to country name.

In [ ]:
#colors_unemployement = ['#fee5d9','#fcbba1','#fc9272','#fb6a4a','#de2d26','#a50f15']
colors_unemployement = ['#fef0d9','#fdd49e','#fdbb84','#fc8d59','#e34a33','#b30000']
min_unemployement_eu = eu_data['2016'].min()
max_unemployement_eu = eu_data['2016'].max()

id_country_mapping = {}

def get_color_eu(country, properties):
    global id_country_mapping
    id_country_mapping[country] = properties['NAME']
    
    values = eu_data.loc[eu_data['info'] == country, '2016'].values
    if len(values) == 0:
        return '#000000'
    
    unemployement_rate = values[0]
    ratio = (unemployement_rate - min_unemployement_eu) / (max_unemployement_eu - min_unemployement_eu)
    index = math.floor(ratio * len(colors_unemployement))
    if index == len(colors_unemployement):
        index = index - 1
    return colors_unemployement[index]

#TODO : Check colors + maybe change intervals

We also define a function that checks if a country is in the dataframe, and if not, we make its overly transparent.

In [ ]:
def get_opacity_eu(country):
    values = eu_data.loc[eu_data['info'] == country, '2016'].values
    return 1 if len(values) > 0 else 0

Now we can create the Europe map, and add the overlay, using the previous functions.

The second map is only used to display the legend.
We used to following website to convert the topojson file to gejson
https://jeffpaine.github.io/geojson-topojson/

In [ ]:
map_eu = folium.Map([51,15], tiles='cartodbpositron', zoom_start=4)


# Color of the country
folium.TopoJson(
    open('topojson/europe.topojson.json'),
    object_path='objects.europe',
    style_function=lambda feature: {
        'fillOpacity' : get_opacity_eu(feature['id']), #opacity for the fill color
        'opacity' : get_opacity_eu(feature['id']), #opacity for the borders
        'fillColor': get_color_eu(feature['id'], feature['properties']),
        'color' : 'black',
        'weight' : 1
        }
).add_to(map_eu)

# 
"""map_eu.choropleth(geo_data='topojson/europe.geojson.json',data=eu_data,
             columns=['info', '2016'],
             key_on='feature.id',
             fill_color='OrRd',
             #fill_color='RdGy',
             fill_opacity=0, 
             line_opacity=0.0,
             legend_name='Percentage of unemployement in country')"""

We used the geopy library to get the position of the center of each country. This position is then used to place a marker with the graph of unemployement.

In [ ]:
"""position_mapping = {}
geolocator = Nominatim()
for country_id in eu_data['info']:
    if country_id in id_country_mapping:
        location = geolocator.geocode(id_country_mapping[country_id])
        if (location):
            position_mapping[country_id] = [location.latitude,location.longitude]

with open('position_mapping.json', 'w') as outfile:
    json.dump(position_mapping, outfile)
    
with open('id_country_mapping.json', 'w') as outfile:
    json.dump(id_country_mapping, outfile)"""

with open('position_mapping.json', 'r') as infile:
    position_mapping = json.load(infile)
    
with open('id_country_mapping.json', 'r') as infile:
    id_country_mapping = json.load(infile)

For each country we create a graph and save it.

In [ ]:
for index, row in eu_data.iterrows():
    global id_country_mapping
    
    if row.values[0] in id_country_mapping:
        row_values = row.iloc[1:].astype(float)
        plot = row_values.transpose().plot()
        plt.ylim(0,30)
        plt.xlabel('Year')
        plt.ylabel('% of unemployed')
        plt.title('Graph of unemployment in ' + id_country_mapping[row.values[0]])
        plt.savefig('eu_graphs/graph' + row.values[0] + '.png')
        plt.close()

For each country we place a custom marker (a black disk) that displays the graph when clicked. 

In [ ]:
all_markers = folium.FeatureGroup("Markers")
for country_id in eu_data['info']:
    if country_id in position_mapping:
        location = position_mapping[country_id]
        if location:
            encoded = base64.b64encode(open('eu_graphs/graph' + country_id + '.png', 'rb').read()).decode()
            html = '<img src="data:image/png;base64,{}" width=450 height=300>'.format
            iframe = folium.IFrame(html(encoded), width=470, height=320)
            popup = folium.Popup(iframe, max_width=2650)

            custom_icon = folium.features.CustomIcon("http://www.i2symbol.com/images/symbols/geometry/black_circle_u25CF_icon_256x256.png", icon_size=(25, 25))
            all_markers.add_child(folium.Marker(location, popup=popup, icon=custom_icon))


map_eu.add_child(all_markers)
map_eu

In [ ]:
ch_data_raw = pd.read_csv('data/unemployed_switzerland.csv')

In [ ]:
groups = ch_data_raw[['Year', 'Canton', 'Unemployment Rate']].groupby(['Canton'])
cantons = ch_data_raw['Canton'].unique()
years = ch_data_raw['Year'].unique()
ch_data = pd.DataFrame(index = cantons, columns=years)

for group, data in groups:
    ch_data.loc[group] = data['Unemployment Rate'].values

ch_data

In [ ]:
colors_unemployement = ['#fef0d9','#fdd49e','#fdbb84','#fc8d59','#e34a33','#b30000']
min_unemployement_ch = ch_data[2016].min()
max_unemployement_ch = ch_data[2016].max()

id_canton_mapping = {}

def get_color_ch(canton, properties):
    global id_canton_mapping
    id_canton_mapping[canton] = properties['name']
    
    #values = ch_data.loc[canton][2016]
    #if len(values) == 0:
    #    return '#000000'
    
    unemployement_rate = ch_data.loc[canton][2016]
    ratio = (unemployement_rate - min_unemployement_ch) / (max_unemployement_ch - min_unemployement_ch)
    index = math.floor(ratio * len(colors_unemployement))
    if index == len(colors_unemployement):
        index = index - 1
    return colors_unemployement[index]

In [ ]:
map_ch = folium.Map([46.8,8.3], tiles='cartodbpositron', zoom_start=8)


# Color of the country
folium.TopoJson(
    open('topojson/ch-cantons.topojson.json'),
    object_path='objects.cantons',
    style_function=lambda feature: {
        'fillOpacity' : 1,#get_opacity_eu(feature['id']), #opacity for the fill color
        'opacity' : 1,#get_opacity_eu(feature['id']), #opacity for the borders
        'fillColor': get_color_ch(feature['id'], feature['properties']),
        'color' : 'black',
        'weight' : 1
        }
).add_to(map_ch)

In [ ]:
"""position_canton_mapping = {}
geolocator = Nominatim()
for canton_id in id_canton_mapping:
    location = geolocator.geocode(id_canton_mapping[canton_id])
    if (location):
        position_canton_mapping[canton_id] = [location.latitude,location.longitude]

with open('position_canton_mapping.json', 'w') as outfile:
    json.dump(position_canton_mapping, outfile)
    
with open('id_canton_mapping.json', 'w') as outfile:
    json.dump(id_canton_mapping, outfile)"""

with open('position_canton_mapping.json', 'r') as infile:
    position_canton_mapping = json.load(infile)
    
with open('id_canton_mapping.json', 'r') as infile:
    id_canton_mapping = json.load(infile)

In [ ]:
for index, row in ch_data.iterrows():
    global id_canton_mapping
    
    if row.values[0] in id_canton_mapping:
        row_values = row.iloc[1:].astype(float)
        plot = row_values.transpose().plot()
        plt.ylim(0,30)
        plt.xlabel('Year')
        plt.ylabel('% of unemployed')
        plt.title('Graph of unemployment in ' + id_canton_mapping[row.values[0]])
        plt.savefig('ch_graphs/graph' + row.values[0] + '.png')
        plt.close()

In [ ]:
all_markers = folium.FeatureGroup("Markers")
for canton_id in id_canton_mapping:
    location = position_canton_mapping[canton_id]
    
    encoded = base64.b64encode(open('ch_graphs/graph' + canton_id + '.png', 'rb').read()).decode()
    html = '<img src="data:image/png;base64,{}" width=450 height=300>'.format
    iframe = folium.IFrame(html(encoded), width=470, height=320)
    #iframe = folium.IFrame("<p>" + id_canton_mapping[canton_id] + "</p>", width=470, height=320)
    popup = folium.Popup(iframe, max_width=2650)

    custom_icon = folium.features.CustomIcon("http://www.i2symbol.com/images/symbols/geometry/black_circle_u25CF_icon_256x256.png", icon_size=(25, 25))
    all_markers.add_child(folium.Marker(location, popup=popup, icon=custom_icon))
        


map_ch.add_child(all_markers)
map_ch